In [1]:
import mysql.connector
import pandas as pd
import csv

# def create_connection():
#     try:
#         connection = mysql.connector.connect(
#             host="localhost",
#             user="root", 
#             password="pw5330", 
#             database="breast_cancer" 
#         )
        
#         if connection.is_connected():
#             print("Connected to MySQL database")
#         return connection
#     except mysql.connector.Error as e:
#         print(f"Error connecting to MySQL: {e}")
#         return None

# connection = create_connection()

In [2]:
import mysql.connector

def create_connection():
    try:
        connection = mysql.connector.connect(
            host="localhost",
            user="root", 
            password="pw5330", 
            database="breast_cancer" 
        )
        if connection.is_connected():
            print("Connected to MySQL database")
        return connection
    except mysql.connector.Error as e:
        print(f"Error connecting to MySQL: {e}")
        return None

def check_and_create_tables(cursor, connection):
    cursor.execute("SHOW TABLES;")
    tables = [table[0].lower() for table in cursor.fetchall()]  # Convert table names to lower case

    # Check if 'cancer_cases' table exists
    if 'cancer_cases' not in tables:
        try:
            create_table_query = """
            CREATE TABLE cancer_cases (
                case_id INT AUTO_INCREMENT PRIMARY KEY,
                diagnosis INT
            );
            """
            cursor.execute(create_table_query)
            connection.commit()  # Commit the transaction
            print("Created table: cancer_cases")
        except mysql.connector.Error as e:
            print(f"Error creating table 'cancer_cases': {e}")
    else:
        print("Table 'cancer_cases' already exists.")
    
    # Check if 'mean_attributes' table exists
    if 'mean_attributes' not in tables:
        try:
            create_table_query = """
            CREATE TABLE mean_attributes (
                case_id INT,
                radius_mean FLOAT,
                texture_mean FLOAT,
                perimeter_mean FLOAT,
                area_mean FLOAT,
                smoothness_mean FLOAT,
                compactness_mean FLOAT,
                concavity_mean FLOAT,
                concave_points_mean FLOAT,
                symmetry_mean FLOAT,
                fractal_dimension_mean FLOAT,
                FOREIGN KEY (case_id) REFERENCES cancer_cases(case_id)
            );
            """
            cursor.execute(create_table_query)
            connection.commit()  # Commit the transaction
            print("Created table: mean_attributes")
        except mysql.connector.Error as e:
            print(f"Error creating table 'mean_attributes': {e}")
    else:
        print("Table 'mean_attributes' already exists.")
    
    # Check if 'worst_attributes' table exists
    if 'worst_attributes' not in tables:
        try:
            create_table_query = """
            CREATE TABLE worst_attributes (
                case_id INT,
                texture_worst FLOAT,
                perimeter_worst FLOAT,
                area_worst FLOAT,
                smoothness_worst FLOAT,
                compactness_worst FLOAT,
                concavity_worst FLOAT,
                concave_points_worst FLOAT,
                symmetry_worst FLOAT,
                fractal_dimension_worst FLOAT,
                FOREIGN KEY (case_id) REFERENCES cancer_cases(case_id)
            );
            """
            cursor.execute(create_table_query)
            connection.commit()  # Commit the transaction
            print("Created table: worst_attributes")
        except mysql.connector.Error as e:
            print(f"Error creating table 'worst_attributes': {e}")
    else:
        print("Table 'worst_attributes' already exists.")

# Main code to create the connection and check the table
connection = create_connection()
if connection:
    cursor = connection.cursor()
    check_and_create_tables(cursor, connection)  # Pass the connection to the function
    cursor.close()
    connection.close()
else:
    print("Failed to connect to the database.")


Connected to MySQL database
Created table: cancer_cases
Created table: mean_attributes
Created table: worst_attributes


# Inserting Data from CSV

In [3]:
def insert_data_from_csv(cursor, df):
    try:
        # Rename columns by replacing spaces with underscores
        df.rename(columns={
            'concave points_mean': 'concave_points_mean',
            'concave points_se': 'concave_points_se',
            'concave points_worst': 'concave_points_worst'
        }, inplace=True)

        # Ensure the 'diagnosis' column is encoded as integer (M -> 1, B -> 0)
        df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})

        # Insert data into 'cancer_cases' table
        for index, row in df.iterrows():
            diagnosis = row['diagnosis']
            cursor.execute("""
            INSERT INTO cancer_cases (diagnosis)
            VALUES (%s)
            """, (diagnosis,))
        
        # Commit the transaction to 'cancer_cases'
        print("Inserted diagnosis data into cancer_cases table.")
        
        # Insert data into 'mean_attributes' table
        for index, row in df.iterrows():
            case_id = index + 1  # Assuming case_id starts from 1 and increments sequentially
            mean_attributes = [
                row['radius_mean'], row['texture_mean'], row['perimeter_mean'], row['area_mean'],
                row['smoothness_mean'], row['compactness_mean'], row['concavity_mean'], row['concave_points_mean'],
                row['symmetry_mean'], row['fractal_dimension_mean']
            ]
            cursor.execute("""
            INSERT INTO mean_attributes (case_id, radius_mean, texture_mean, perimeter_mean, area_mean,
                                          smoothness_mean, compactness_mean, concavity_mean, concave_points_mean,
                                          symmetry_mean, fractal_dimension_mean)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (case_id, *mean_attributes))
        
        # Commit the transaction to 'mean_attributes'
        print("Inserted mean attributes data into mean_attributes table.")
        
        # Insert data into 'worst_attributes' table
        for index, row in df.iterrows():
            case_id = index + 1  # Assuming case_id starts from 1 and increments sequentially
            worst_attributes = [
                row['texture_worst'], row['perimeter_worst'], row['area_worst'], row['smoothness_worst'],
                row['compactness_worst'], row['concavity_worst'], row['concave_points_worst'], row['symmetry_worst'],
                row['fractal_dimension_worst']
            ]
            cursor.execute("""
            INSERT INTO worst_attributes (case_id, texture_worst, perimeter_worst, area_worst, smoothness_worst,
                                          compactness_worst, concavity_worst, concave_points_worst, symmetry_worst,
                                          fractal_dimension_worst)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (case_id, *worst_attributes))
        
        # Commit the transaction to 'worst_attributes'
        print("Inserted worst attributes data into worst_attributes table.")
        
        # Commit all the changes at once
        connection.commit()
        print("Data insertion completed.")
    
    except mysql.connector.Error as e:
        print(f"Error inserting data: {e}")
        connection.rollback()  # Rollback in case of error


In [4]:
# Main code to create the connection and insert the data
connection = create_connection()
if connection:
    cursor = connection.cursor()
    
    # Load the CSV file into a pandas DataFrame
    df = pd.read_csv('../Data/Raw/breast-cancer.csv')  # Replace with your file path
    
    # Insert the data from the DataFrame into the database
    insert_data_from_csv(cursor, df)
    
    cursor.close()
    connection.close()
else:
    print("Failed to connect to the database.")

Connected to MySQL database
Inserted diagnosis data into cancer_cases table.
Inserted mean attributes data into mean_attributes table.
Inserted worst attributes data into worst_attributes table.
Data insertion completed.
